In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
sys.path.insert(0, '../scripts')
from l5kit_modified.l5kit_modified import FramesDataset, create_chopped_dataset, AgentDatasetModified, get_agent_indices_set
from kalman import KalmanTrackerPredictor
from lane_processing import *
import pickle
from l5kit.evaluation import write_pred_csv, compute_metrics_csv
from l5kit.evaluation.metrics import neg_multi_log_likelihood
from l5kit.data import LocalDataManager, ChunkedDataset
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

from pytz import timezone
from datetime import datetime, timedelta
from l5kit.data.filter import filter_tl_faces_by_status
from workalendar.usa import CaliforniaSanFrancisco
import pandas as pd
import json
from django.core.serializers.json import DjangoJSONEncoder

from collections import defaultdict
import gc
import multiprocessing as mp
import tracemalloc
import linecache
from functools import partial
tracemalloc.start()

def display_top(snapshot, key_type='lineno', limit=20):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))



snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

%matplotlib inline
    
os.environ["L5KIT_DATA_FOLDER"] = "../input/"

import zarr

Top 20 lines
#1: python3.7/codeop.py:141: 7.2 KiB
    codeob = compile(source, filename, symbol, self.flags, 1)
#2: <ipython-input-2-ca8240b77984>:62: 0.4 KiB
    snapshot = tracemalloc.take_snapshot()
#3: core/interactiveshell.py:3417: 0.4 KiB
    exec(code_obj, self.user_global_ns, self.user_ns)
#4: <ipython-input-2-ca8240b77984>:35: 0.1 KiB
    def display_top(snapshot, key_type='lineno', limit=20):
#5: core/interactiveshell.py:3337: 0.1 KiB
    if (await self.run_code(code, result,  async_=asy)):
#6: core/interactiveshell.py:117: 0.1 KiB
    Module = lambda nodelist, type_ignores: OriginalModule(nodelist)
#7: core/compilerop.py:150: 0.0 KiB
    self.flags &= ~turn_on_bits
#8: core/interactiveshell.py:3331: 0.0 KiB
    mod = Module([node], [])
Total allocated size: 8.4 KiB


In [3]:
lane_seq_df = pd.read_hdf('../input/lane_seq_df.hdf5', key='data')

In [4]:
pd.set_option('display.max_colwidth', 500)
lane_seq_df.head()

,track_id,hist_seq,future_seq
0,328,"((bUGp, 0.2222222222222222, 2, 22.188255, 0.033309937, 0, nan, nan), (sRCY, 0.4444444444444444, 1, 22.307493, 0.0, 0, nan, nan))","(bUGp, sRCY, O5jj)"
1,328,"((bUGp, 0.2222222222222222, 2, 22.188255, 0.033309937, 0, nan, nan), (sRCY, 0.4444444444444444, 1, 22.307493, 0.0, 0, nan, nan), (O5jj, 0.7692307692307693, 2, 22.45031, 0.04981613, 0, nan, nan))","(bUGp, sRCY, O5jj, sRCY)"
2,328,"((bUGp, 0.2222222222222222, 2, 22.188255, 0.033309937, 0, nan, nan), (sRCY, 0.4444444444444444, 1, 22.307493, 0.0, 0, nan, nan), (O5jj, 0.7692307692307693, 2, 22.45031, 0.04981613, 0, nan, nan))","(bUGp, sRCY, O5jj, sRCY)"
3,328,"((bUGp, 0.2222222222222222, 2, 22.188255, 0.033309937, 0, nan, nan), (sRCY, 0.4444444444444444, 1, 22.307493, 0.0, 0, nan, nan), (O5jj, 0.7692307692307693, 2, 22.45031, 0.04981613, 0, nan, nan))","(bUGp, sRCY, O5jj, sRCY)"
4,328,"((bUGp, 0.2222222222222222, 2, 22.188255, 0.033309937, 0, nan, nan), (sRCY, 0.4444444444444444, 1, 22.307493, 0.0, 0, nan, nan), (O5jj, 0.7692307692307693, 2, 22.45031, 0.04981613, 0, nan, nan))","(bUGp, sRCY, O5jj, sRCY)"


In [5]:
np.array(lane_seq_df['hist_seq'].values[0])[0]

array(['bUGp', '0.2222222222222222', '2', '22.188255', '0.033309937', '0',
       'nan', 'nan'], dtype='<U18')

In [6]:
lane_seq_df['hist_len']  = lane_seq_df['hist_seq'].map(len)
lane_seq_df['future_len']  = lane_seq_df['future_seq'].map(len)

In [7]:
lane_seq_df['hist_len'].sum(), lane_seq_df['future_len'].sum()

(17789881, 24486247)

In [57]:
DATAPOINT_DTYPE = [
    ("track_id", np.int64),  
    ("hist_index_interval", np.int64, (2,)),
    ("future_index_interval", np.int64, (2,)),
]


HIST_DTYPE = [
    ("lane_id", "<U8"),
    ("lane_completion_at_start", np.float32),
    ("number_of_car_observations_on_line", np.int16),
    ("car_mean_speed_on_line", np.float32),
    ("car_std_speed_on_line", np.float32),    
    ("number_of_other_car_observations_on_line", np.int16),
    ("other_cars_mean_speed_on_line", np.float32),
    ("other_cars_std_speed_on_line", np.float32),
]

FUTURE_DTYPE = [
    ("lane_id", "<U8"),
]

In [60]:
lane_dataset_root = zarr.open_group('../input/lane_seqs.zarr', mode='w')
chunk_size = 5000
datapoints = lane_dataset_root.require_dataset(
            'datapoints', dtype=DATAPOINT_DTYPE, chunks=(chunk_size,), shape=(len(lane_seq_df),)
        )
hist_seqs = lane_dataset_root.require_dataset(
    'hist_seqs', dtype=HIST_DTYPE, chunks=(chunk_size,), shape=(lane_seq_df['hist_len'].sum(),)
)
future_seqs = lane_dataset_root.require_dataset(
    'future_seqs', dtype=FUTURE_DTYPE, chunks=(chunk_size,), shape=(lane_seq_df['future_len'].sum(),)
)

In [61]:
hist_seq_idx = 0
future_seq_idx = 0
chunk_datapoints_i = 0
chunk_datapoints_size = 0
chunk_datapoints = np.ndarray(shape=(chunk_size,), dtype=DATAPOINT_DTYPE)
chunk_hist_i = 0
chunk_hist_size = 0
chunk_hist = np.ndarray(shape=(chunk_size,), dtype=HIST_DTYPE)
chunk_future_i = 0
chunk_future_size = 0
chunk_future = np.ndarray(shape=(chunk_size,), dtype=FUTURE_DTYPE)

for row_idx, row in tqdm(lane_seq_df.iterrows(), total=len(lane_seq_df)):
    chunk_datapoints[chunk_datapoints_size] = ((row['track_id'], (hist_seq_idx, hist_seq_idx + len(row['hist_seq'])), (future_seq_idx, future_seq_idx + len(row['future_seq']))))
    chunk_datapoints_size += 1
    if chunk_datapoints_size % chunk_size == 0:
        datapoints[chunk_datapoints_i*chunk_size: chunk_size*(chunk_datapoints_i + 1)] = chunk_datapoints
        chunk_datapoints = np.ndarray(shape=(chunk_size,), dtype=DATAPOINT_DTYPE)
        chunk_datapoints_size = 0
        chunk_datapoints_i += 1        
    
    for hist_i, hist_tuple in enumerate(row['hist_seq']):
        chunk_hist[chunk_hist_size] = hist_tuple
        chunk_hist_size += 1
        if chunk_hist_size % chunk_size == 0:
            hist_seqs[chunk_hist_i*chunk_size: chunk_size*(chunk_hist_i + 1)] = chunk_hist
            chunk_hist_size = 0
            chunk_hist = np.ndarray(shape=(chunk_size,), dtype=HIST_DTYPE)
            chunk_hist_i += 1
            
    for future_i, future_line in enumerate(row['future_seq']):
        chunk_future[chunk_future_size] = future_line
        chunk_future_size += 1
        if chunk_future_size % chunk_size == 0:
            future_seqs[chunk_future_i*chunk_size: chunk_size*(chunk_future_i + 1)] = chunk_future
            chunk_future_size = 0
            chunk_future = np.ndarray(shape=(chunk_size,), dtype=FUTURE_DTYPE)
            chunk_future_i += 1
    hist_seq_idx += len(row['hist_seq'])
    future_seq_idx += len(row['future_seq'])
    
                                   